In [1]:
!pip install wikidata
!pip install networkx
!pip install pyvis
!pip install datasets
from huggingface_hub import login

HuggingFace Access for the dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
from huggingface_hub import notebook_login

# token = ""

# non spuntare add token as git credential
notebook_login()

In [4]:
import sys
sys.path.append("/content/drive/MyDrive/NLP_HOMEWORK_1/SocioEmbeddings/utils/")
drive_root = "/content/drive/MyDrive/NLP_HOMEWORK_1/SocioEmbeddings/"
data_folder = drive_root+"data/"
dataset_folder = drive_root + "csv_datasets/"

# Dataset loading
We created an architecture that helps the end-user to easily load datasets from HuggingFace or CSV Files and easily make predictions. (all the files used are in `utils/`)<br>

In [5]:
from dataset_loader import DatasetLoader
csv_path = "/content/drive/MyDrive/NLP_HOMEWORK_1/data/gold_dataset.tsv" # use sep = "\t" as extra parameter
hf_path = "sapienzanlp/nlp2025_hw1_cultural_dataset"

dataset_type = "csv" # "csv"
unlabeled_test_folder = f"{dataset_folder}test_unlabeled.csv"

dataset = DatasetLoader(dataset_type=dataset_type, path=unlabeled_test_folder, sep=",").load()

When using the models, the Wikipedia text is scraped automatically, but since the APIs have rate limiting, we needed to slow down that process using `time.sleep(1)` so processing each Wikipedia article requires more than 1 second. <br>
This is quite slow: we saved the file in local so that there's no need to do that process again. <br>
Of course if you still want to scrape by yourself, just dont pass wikidict to the Predict constructor.

In [6]:
import pickle
with open(data_folder+"TEST_WIKIDICT.pkl", "rb") as f:
    wikidict = pickle.load(f)

Running this multiple times can cause RAM issues.

#LANGUAGE MODEL SOLUTION

If you want to scrape from wikidata, put:
<br>
wikidict = None. Instead with wikidict=wikidict, the it will load a previous download


In [7]:
from predictor import Predict

pred = Predict(model_type="lm", dataset=dataset, data_folder=data_folder, wikidict=wikidict)

Creating Wikipedia articles dictionary


100%|██████████| 300/300 [06:14<00:00,  1.25s/it]


In [14]:
res_lm = pred.predict(drive_root+"predictions/socioembeddings_output_lm.csv")

100%|██████████| 300/300 [00:20<00:00, 14.36it/s]



Predictions saved to /content/drive/MyDrive/NLP_HOMEWORK_1/SocioEmbeddings/predictions/socioembeddings_output_lm.csv


In [10]:
res_lm

,item,name,description,type,category,subcategory,label
0,http://www.wikidata.org/entity/Q2427430,Northeast Flag Replacement,Zhang Xueliang's announcement on 29 December 1...,concept,History,historical event,cultural exclusive
1,http://www.wikidata.org/entity/Q125482,imam,Islamic leadership position,concept,philosophy and religion,religious leader,cultural representative
2,http://www.wikidata.org/entity/Q15789,FC Bayern Munich,"association football club in Munich, Germany",named entity,sports,sports club,cultural representative
3,http://www.wikidata.org/entity/Q582496,Fome Zero,program intended to eradicate hunger and extre...,named entity,politics,government agency,cultural exclusive
4,http://www.wikidata.org/entity/Q572811,Anthony Award,awards given at Bouchercon for mystery literature,named entity,Literature,literary award,cultural exclusive
...,...,...,...,...,...,...,...
295,http://www.wikidata.org/entity/Q4878968,bed skirt,bedding accessory consisting of a flat and gat...,entity,Fashion,textile,cultural agnostic
296,http://www.wikidata.org/entity/Q1361932,family film,film genre; films intended for a family audien...,entity,Films,film genre,cultural agnostic
297,http://www.wikidata.org/entity/Q639669,musician,"person who composes, conducts or performs music",concept,Visual Arts,artist,cultural agnostic
298,http://www.wikidata.org/entity/Q616077,VJing,broad designation for realtime visual performance,concept,Visual Arts,visual arts,cultural agnostic


#HYBRID SOLUTION


If you want to scrape from wikidata, put:
<br>
wikidict = None.
Instead with wikidict=wikidict, the it will load a previous download

In [11]:
pred_nonlm = Predict(model_type="nonlm", dataset=dataset, data_folder=data_folder, wikidict=wikidict)

Creating Wikidata graph structure
Extracting Level 1...


100%|██████████| 300/300 [00:07<00:00, 42.23it/s]


Extracting Level 2 from 3108 unique entity values...


 74%|███████▍  | 2298/3108 [00:56<00:19, 42.11it/s]

Error on E225: 


...

Error on E153: 


 83%|████████▎ | 2595/3108 [01:04<00:11, 43.88it/s]

Error on E371: 


100%|██████████| 3108/3108 [01:16<00:00, 40.62it/s]


Preprocessing data
Retrieving the model


In [16]:
res_nonlm = pred_nonlm.predict(drive_root+"predictions/socioembeddings_output_nonlm.csv")

100%|██████████| 300/300 [00:00<00:00, 25109.58it/s]


Predictions saved to /content/drive/MyDrive/NLP_HOMEWORK_1/SocioEmbeddings/predictions/socioembeddings_output_nonlm.csv


In [13]:
res_nonlm

,item,name,description,type,category,subcategory,label
0,http://www.wikidata.org/entity/Q2427430,Northeast Flag Replacement,Zhang Xueliang's announcement on 29 December 1...,concept,History,historical event,cultural representative
1,http://www.wikidata.org/entity/Q125482,imam,Islamic leadership position,concept,philosophy and religion,religious leader,cultural representative
2,http://www.wikidata.org/entity/Q15789,FC Bayern Munich,"association football club in Munich, Germany",named entity,sports,sports club,cultural representative
3,http://www.wikidata.org/entity/Q582496,Fome Zero,program intended to eradicate hunger and extre...,named entity,politics,government agency,cultural representative
4,http://www.wikidata.org/entity/Q572811,Anthony Award,awards given at Bouchercon for mystery literature,named entity,Literature,literary award,cultural representative
...,...,...,...,...,...,...,...
295,http://www.wikidata.org/entity/Q4878968,bed skirt,bedding accessory consisting of a flat and gat...,entity,Fashion,textile,cultural agnostic
296,http://www.wikidata.org/entity/Q1361932,family film,film genre; films intended for a family audien...,entity,Films,film genre,cultural agnostic
297,http://www.wikidata.org/entity/Q639669,musician,"person who composes, conducts or performs music",concept,Visual Arts,artist,cultural agnostic
298,http://www.wikidata.org/entity/Q616077,VJing,broad designation for realtime visual performance,concept,Visual Arts,visual arts,cultural agnostic
